In [15]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

In [16]:
data_train = pd.read_csv('XES3G5M/kc_level/train_valid_sequences.csv')
data_test = pd.read_csv('XES3G5M/kc_level/test.csv')

with open(f'XES3G5M/metadata/questions.json', 'r') as file:
    q_data = json.load(file)

with open(f'XES3G5M/metadata/kc_routes_map.json', 'r') as file:
    route_data = json.load(file)

In [17]:
data_train.head()

,fold,uid,questions,concepts,responses,timestamps,selectmasks,is_repeat
0,0,11066,"3751,3752,3753,3754,1990,3739,3740,3742,3756,3...","187,187,374,187,374,188,188,228,166,170,221,40...","1,1,1,0,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,...","1595229836000,1595233013000,1595233687000,1595...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
1,0,11066,"95,4294,4294,4300,4300,4301,4302,4308,115,130,...","63,64,102,659,376,147,147,65,64,78,81,468,78,7...","0,1,1,1,1,1,0,0,1,1,0,1,1,1,1,1,1,1,1,0,0,0,1,...","1608890160000,1608892236000,1608892236000,1608...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
2,0,14482,"266,268,2203,870,2316,2587,2323,2588,5194,868,...","9,139,140,9,496,232,231,231,146,145,231,31,65,...","0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,...","1595896054000,1595896054000,1595896054000,1595...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,..."
3,0,14482,"1280,1046,405,404,404,4623,4622,4622,4624,4954...","257,32,204,204,205,206,526,206,206,206,206,207...","0,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,0,0,1,1,1,1,...","1614471945000,1614471945000,1614471945000,1614...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
4,0,10532,"268,266,2203,2588,2587,2323,275,811,2205,2205,...","139,9,140,231,232,231,30,30,62,65,62,65,152,15...","1,0,0,0,1,0,1,1,1,1,1,0,1,0,1,0,0,0,1,1,1,0,1,...","1595918717000,1595918717000,1595918717000,1596...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,..."


In [18]:
response_data = dict()
stu_response_data = dict() 
stu_num = np.random.choice(np.arange(18066), size=10000, replace=False)
least_respone_num = 300

for stu in tqdm(stu_num, desc='Filter student'):
    for data in data_train.loc[data_train['uid'] == stu].values:
        uid, questions, responses, selectmasks = data[1], data[2].split(','), data[4].split(','), data[6].split(',')
        for i in range(len(questions)):
            if selectmasks[i] == '-1':
                break
            # if questions[i] == -1:
            #     print(questions[i])
            tmp_data = (uid, questions[i])
            response_data[tmp_data] = responses[i]
    
    for data in data_test.loc[data_test['uid'] == stu].values:
        uid, questions, responses = data[1], data[2].split(','), data[4].split(',')
        for i in range(len(questions)):
            tmp_data = (uid, questions[i])
            response_data[tmp_data] = responses[i]

for key, value in tqdm(response_data.items(), desc='Filter the least respone number'):
    if key[0] not in stu_response_data:
        stu_response_data[key[0]] = []
    stu_response_data[key[0]].append([int(key[0]), int(key[1]), int(value)])

Filter the least respone number: 100%|██████████| 2944842/2944842 [00:02<00:00, 1013467.03it/s]


In [19]:
TotalData = []
stu_map = dict()
cnt_stu = 0
question_set = set()
cnt_question = 0
question_map = dict()
concept_set = set()
cnt_concept = 0
concept_map = dict()

for key, value in tqdm(stu_response_data.items(), desc='Remap student_id, question_id and concept_id'):
    if len(value) >= least_respone_num:
        stu_map[key] = cnt_stu
        cnt_stu += 1
        for data in value:
            question_set.add(data[1])
            for concept in q_data[str(data[1])]['kc_routes'][0].split('----'):
                concept_set.add(concept)

for question in question_set:
    question_map[question] = cnt_question
    cnt_question += 1

for concept in concept_set:
    concept_map[concept] = cnt_concept
    cnt_concept += 1

Remap student_id, question_id and concept_id: 100%|██████████| 10000/10000 [00:01<00:00, 5051.71it/s]


In [20]:
TotalData = []
q_matrix = np.zeros((cnt_question, cnt_concept))

for key, value in tqdm(stu_response_data.items(), desc='Construct final data'):
    if len(value) >= least_respone_num:
        for data in value:
            TotalData.append([stu_map[data[0]], question_map[data[1]], data[2]])
            for concept in q_data[str(data[1])]['kc_routes'][0].split('----'):
                q_matrix[question_map[data[1]]][concept_map[concept]] = 1

Construct final data: 100%|██████████| 10000/10000 [00:05<00:00, 1855.23it/s]


In [21]:
 print('Final student number: {}, Final question number: {}, Final concept number: {}, Final response number: {}'.format(cnt_stu, cnt_question, cnt_concept, len(TotalData)))

Final student number: 3971, Final question number: 6475, Final concept number: 1029, Final response number: 1512280
